In [1]:
%config Completer.use_jedi = False
import numpy as np
import pandas as pd

In [4]:
features = np.load('/data/hookbill/hadfield/syntheticVS/data/zinc_50ops_ac0025_t4_processed/features_plec_45.npy', allow_pickle=True)
labels = np.load('/data/hookbill/hadfield/syntheticVS/data/zinc_50ops_ac0025_t4_processed/labels_plec_45.npy', allow_pickle=True)



In [7]:
print(features.shape[0], labels.shape[0])
print(sum(labels))

10000 10000
5018


In [26]:
def control_prop_actives(features, labels, additional_features = None, prop_actives = None):
    
    #additional_features is included in case we have to do the same subsetting with 
    #a different type of feature set - e.g. a Morgan Fingerprint.
    
    if prop_actives is None:
        return features, labels #i.e. don't change the proportion at all
    elif prop_actives < 0 or prop_actives > 1:
            raise ValueError('Please provide a proportion between 0 and 1.')
    else:
        
        initial_num_actives = sum(labels == 1)
        initial_num_inactives = sum(labels == 0)
        
        if additional_features is None:
            df = pd.DataFrame({'labels':labels, 'features':[x for x in features]})
        else:
            df = pd.DataFrame({'labels':labels, 'features':[x for x in features], 'add_features':[x for x in additional_features]})
            
            
        df_inactives = df.loc[df['labels'] == 0]
        df_actives = df.loc[df['labels'] == 1]

        if initial_num_actives*(1/prop_actives - 1) < initial_num_inactives:
            #We can use all the existing actives and just subset the inactives
            num_to_sample = initial_num_actives * (1/prop_actives - 1)
            df_inactives_subset = df_inactives.sample(n = int(np.floor(num_to_sample)), random_state = 100)
            df_combined_subset = df_actives.append(df_inactives_subset)
        
        else:
            num_to_sample = initial_num_inactives*(prop_actives/(1 - prop_actives))
            df_actives_subset = df_actives.sample(n = int(np.floor(num_to_sample)), random_state = 100)
            df_combined_subset = df_inactives.append(df_actives_subset)
            
            
        #shuffle order
        df_combined_subset = df_combined_subset.sample(frac=1)
        subset_labels = np.array(df_combined_subset['labels'])
        subset_features = np.array(list(df_combined_subset['features']))
        subset_additional_features = np.array(list(df_combined_subset['add_features']))
       
    
        if additional_features is None:
            return subset_features, subset_labels
        else:
            return subset_features, subset_labels, additional_features
            
        
        
    
    
    
            

In [31]:
sf, sl = control_prop_actives(features, labels, prop_actives=0.69)

In [32]:
sum(sl)/sl.shape[0]

0.69004400440044

In [9]:
sum(labels == 1)

5018

In [13]:
df = pd.DataFrame({'labels':labels, 'features':[x for x in features]})
df_inactives = df.loc[df['labels'] == 0]
df_inactives.sample(n = 10)

,labels,features
3981,0,"[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6628,0,"[0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8415,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5804,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1524,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4481,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7819,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2850,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2490,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8010,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


In [17]:
np.array(list(df['features'])).shape

(10000, 1024)

In [19]:
features.shape

(10000, 1024)